In [1]:
from sklearn.externals import joblib
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
LinearModel = pickle.load(open('linear_model.sav', 'rb')) #lr
KnnModel = pickle.load(open('knnModel.sav', 'rb')) #knn

In [3]:
NewBaseModel = load_model('NewBaseModel.h5') #regr
LargerModel = load_model('LargerModel.h5') #lrg_model
baseModel = load_model('baseline.h5') #baseline

In [4]:
data = pd.read_csv('openpowerlifting2.csv')
meets = pd.read_csv('meets2.csv')
AthID =pd.read_csv('AthTable.csv')
model_data=pd.read_csv('AthleteResults.csv')

In [5]:
model_data.index.name = 'AthID'

In [6]:
model_data.head()

,First_Sex,First_Equipment,First_BodyweightKg,First_Wilks,First_WhereHeld,First_EventDate,First_Competition_experience,Second_Sex,Second_Equipment,Second_BodyweightKg,...,Second_EventDate,Second_Competition_experience,Third_Sex,Third_Equipment,Third_BodyweightKg,Third_Wilks,Third_WhereHeld,Third_EventDate,Third_Competition_experience,birth_year
AthID,,,,,,,,,,,,,,,,,,,,,
0,1,3,100.0,418.40,SoPac,2018-01-27,1,1,1,87.20,...,2017-09-30,2,1,3,100.00,413.84,SoPac,2017-09-09,3,1992.0
1,1,3,99.8,493.35,SoPac,2018-01-27,1,1,3,102.60,...,2017-09-09,2,1,3,99.50,500.05,SoPac,2017-07-21,3,1989.0
2,1,3,98.8,334.84,SoPac,2018-01-27,1,1,1,99.30,...,2017-10-07,2,1,3,93.80,340.99,SoPac,2012-11-09,3,1968.0
3,1,3,98.0,395.80,SoPac,2018-01-27,1,1,3,89.80,...,2016-07-08,2,1,3,89.80,399.45,SoPac,2014-11-21,3,1969.0
4,1,3,89.7,380.49,SoPac,2018-01-27,1,1,2,74.39,...,2017-02-23,2,1,2,74.57,306.66,NAmerica,2017-02-18,3,1990.0


In [7]:
AthID.drop_duplicates(subset='AthID', keep='first', inplace=True)

In [8]:
##
#df = pd.merge(data, AthID, on='Name')
#df = pd.merge(AthID, data, on='Name')

In [9]:
data.shape

(491167, 17)

In [10]:
meets['year'] = meets['Date'].str.split('/').str.get(2).astype(int)
pd.to_datetime(meets['Date'])
meets.query('year == 2018')# & Date >=3/1/2018')

,MeetID,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName,year
14,14,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018
139,139,apf/1742,APF,2/3/2018,USA,OH,NaN,Ohio Cup,2018
141,141,apf/1801,APF,2/10/2018,USA,SD,NaN,SD State Powerlifting Championships,2018
142,142,apf/1802,APF,2/24/2018,USA,OH,NaN,Buckeye Brawl,2018
143,143,apf/1803,APF,2/17/2018,USA,IL,NaN,Ernie Frantz Squat Invitational,2018
144,144,apf/1804,APF,3/10/2018,USA,MI,Grand Rapids,Michigan State Meet,2018
145,145,apf/1805,APF,3/17/2018,USA,NaN,NaN,Armed Forces Meet,2018
233,233,bp/1801,BP,1/21/2018,UK,NaN,Horncastle,British Equipped Bench Press Championships,2018
234,234,bp/1802,BP,1/20/2018,UK,NaN,Horncastle,British Equipped Powerlifting Championships,2018
235,235,bp/1803,BP,2/18/2018,UK,NaN,Portsmouth,British Classic Bench Press Championships,2018


In [11]:
data.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,NaN,20.41,NaN,70.31,138.35,1,155.05
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47


In [12]:
pd.to_datetime(meets['Date'])
twenty18 = meets.query('year==2018')

In [13]:
#take New data, add AthID, drop anyone not in model data.
#add in meetinformation. Drop any not registered for meet we wish to predict.


In [14]:
twenty18 = pd.merge(data, twenty18, on='MeetID')
twenty18.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,...,Place,Wilks,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName,year
0,14,Becca Dison,F,Raw,19.0,Open,55.91,56,NaN,125.0,...,1,403.50,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018
1,14,Andrea Fox,F,Raw,35.0,Open,67.22,67.5,NaN,102.5,...,1,322.46,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018
2,14,Candace Locklear,F,Raw,23.0,Open,74.66,75,NaN,152.5,...,1,378.96,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018
3,14,Jennifer Broome,F,Wraps,35.0,Open,73.84,75,NaN,112.5,...,1,312.03,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018
4,14,Tara Peele,F,Raw,41.0,M1-Mst,73.66,75,NaN,115.0,...,1,307.71,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018


In [15]:
twenty18 = pd.merge(twenty18, AthID, on='Name')

In [16]:
twenty18.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,...,Wilks,MeetPath,Federation,Date,MeetCountry,MeetState,MeetTown,MeetName,year,AthID
0,14,Jennifer Broome,F,Wraps,35.0,Open,73.84,75,NaN,112.5,...,312.03,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018,45743
1,14,Janet Martin,F,Raw,49.0,M2-Mst,113.85,90+,NaN,NaN,...,155.43,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018,43375
2,14,Janet Martin,F,Raw,49.0,M2-Mst,113.85,90+,NaN,NaN,...,50.47,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018,43375
3,14,Janet Martin,F,Raw,49.0,M2-Mst,113.85,90+,NaN,NaN,...,104.97,365strong/1801,365Strong,3/10/2018,USA,NC,Monroe,Enterprise Power Revolution,2018,43375
4,1319,Janet Martin,F,Raw,NaN,Master 2,83.40,84,NaN,110.0,...,288.63,cpu/2018-02-19-ada60117,CPU,2/19/2018,Canada,AB,Calgary,Canadian Championship,2018,43375


In [17]:
twenty18.isnull().sum()

MeetID                0
Name                  0
Sex                   0
Equipment             0
Age                7244
Division            122
BodyweightKg          1
WeightClassKg       152
Squat4Kg          10721
BestSquatKg        1249
Bench4Kg          10707
BestBenchKg         521
Deadlift4Kg       10688
BestDeadliftKg      802
TotalKg             355
Place                 1
Wilks               355
MeetPath              0
Federation            0
Date                  0
MeetCountry           0
MeetState          1140
MeetTown           4533
MeetName              0
year                  0
AthID                 0
dtype: int64

In [18]:
twenty18.columns

Index(['MeetID', 'Name', 'Sex', 'Equipment', 'Age', 'Division', 'BodyweightKg',
       'WeightClassKg', 'Squat4Kg', 'BestSquatKg', 'Bench4Kg', 'BestBenchKg',
       'Deadlift4Kg', 'BestDeadliftKg', 'TotalKg', 'Place', 'Wilks',
       'MeetPath', 'Federation', 'Date', 'MeetCountry', 'MeetState',
       'MeetTown', 'MeetName', 'year', 'AthID'],
      dtype='object')

In [19]:
to_keep = ['AthID', 'Name', 'Date', 'MeetCountry', 'Wilks', 'BodyweightKg', 'Equipment', 'Sex', 'Age', 'year']

In [20]:
twenty18_kept=twenty18[to_keep]
twenty18_kept['duplicates'] = twenty18_kept.groupby(['AthID', 'Date']).cumcount()+1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
##data['First_WhereHeld'] = LabelEncoder().fit_transform(data['First_WhereHeld'])
##data['Second_WhereHeld'] = LabelEncoder().fit_transform(data['Second_WhereHeld'])
##data['Third_WhereHeld'] = LabelEncoder().fit_transform(data['Third_WhereHeld'])

In [22]:
twenty18_kept.head()

,AthID,Name,Date,MeetCountry,Wilks,BodyweightKg,Equipment,Sex,Age,year,duplicates
0,45743,Jennifer Broome,3/10/2018,USA,312.03,73.84,Wraps,F,35.0,2018,1
1,43375,Janet Martin,3/10/2018,USA,155.43,113.85,Raw,F,49.0,2018,1
2,43375,Janet Martin,3/10/2018,USA,50.47,113.85,Raw,F,49.0,2018,2
3,43375,Janet Martin,3/10/2018,USA,104.97,113.85,Raw,F,49.0,2018,3
4,43375,Janet Martin,2/19/2018,Canada,288.63,83.40,Raw,F,NaN,2018,1


In [23]:
mapping = {'Wraps':3, 'Raw':1, 'Multi-ply': 0, 'Single-ply':2}
twenty18_kept.replace({'Equipment':mapping}, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [24]:
twenty18_kept = twenty18_kept.query('duplicates==1')

In [25]:

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('USA', 'NAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Canada', 'NAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('US Virgin Islands', 'NAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Brazil', 'NAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Colombia', 'SAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Cayman Islands', 'NAmerica')

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Peru', 'SAmerica')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Puerto Rico', 'NAmerica')


twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('UK', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('England', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Ireland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Czechia', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Germany', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Norway', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Sweden', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Scotland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Poland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Finland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('N.Ireland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Ukraine', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Luxembourg', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('N.Europe', 'Europe')

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Iceland', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Estonia', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Serbia', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Italy', 'Europe')

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Belarus', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Spain', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('France', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Denmark', 'Europe')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Wales', 'Europe')

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Singapore', 'Asia')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Russia', 'Asia')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Israel', 'Asia')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Malaysia', 'Asia')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Kazakhstan', 'Asia')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('South Africa', 'Africa')

twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('Australia', 'SoPac')
twenty18_kept['MeetCountry'] = twenty18_kept['MeetCountry'].str.replace('New Zealand', 'SoPac')

In [26]:
twenty18_kept.head()

,AthID,Name,Date,MeetCountry,Wilks,BodyweightKg,Equipment,Sex,Age,year,duplicates
0,45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,F,35.0,2018,1
1,43375,Janet Martin,3/10/2018,NAmerica,155.43,113.85,1,F,49.0,2018,1
4,43375,Janet Martin,2/19/2018,NAmerica,288.63,83.40,1,F,NaN,2018,1
5,42429,James Corey McManus,3/10/2018,NAmerica,206.72,77.84,1,M,37.0,2018,1
6,67074,Matt Borquez,3/10/2018,NAmerica,280.78,80.38,1,M,29.0,2018,1


In [27]:
twenty18_kept.rename(columns={'MeetCountry':'WhereHeld'}, inplace=True)
#twenty18_kept['WhereHeld']= LabelEncoder().fit_transform(twenty18_kept['WhereHeld'])
twenty18_kept.drop('duplicates', axis=1, inplace=True)

In [28]:
twenty18_kept.head()

,AthID,Name,Date,WhereHeld,Wilks,BodyweightKg,Equipment,Sex,Age,year
0,45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,F,35.0,2018
1,43375,Janet Martin,3/10/2018,NAmerica,155.43,113.85,1,F,49.0,2018
4,43375,Janet Martin,2/19/2018,NAmerica,288.63,83.40,1,F,NaN,2018
5,42429,James Corey McManus,3/10/2018,NAmerica,206.72,77.84,1,M,37.0,2018
6,67074,Matt Borquez,3/10/2018,NAmerica,280.78,80.38,1,M,29.0,2018


In [29]:
twenty18_kept.dropna(axis=0, inplace=True)
twenty18_kept['Competition_experience'] = twenty18_kept.groupby(['AthID']).cumcount()+1

twenty18_kept['birth_year'] = twenty18_kept['year'] - twenty18_kept['Age']

twenty18_kept['Sex'] = np.where(twenty18_kept['Sex']=='M', '1', '0')
twenty18_kept.rename(columns={'Date':'EventDate'}, inplace=True)

In [30]:
twenty18_kept = twenty18_kept.query('AthID!=43375')
twenty18_kept = twenty18_kept.query('AthID!= 63523')
twenty18_kept.head()

,AthID,Name,EventDate,WhereHeld,Wilks,BodyweightKg,Equipment,Sex,Age,year,Competition_experience,birth_year
0,45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,0,35.0,2018,1,1983.0
5,42429,James Corey McManus,3/10/2018,NAmerica,206.72,77.84,1,1,37.0,2018,1,1981.0
6,67074,Matt Borquez,3/10/2018,NAmerica,280.78,80.38,1,1,29.0,2018,1,1989.0
7,9600,Avi Bagley,3/10/2018,NAmerica,363.88,79.29,1,1,28.0,2018,1,1990.0
8,45082,Jeff Brown,3/10/2018,NAmerica,387.28,98.52,3,1,33.0,2018,1,1985.0


In [31]:
twenty18_kept.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2769 entries, 0 to 10623
Data columns (total 12 columns):
AthID                     2769 non-null int64
Name                      2769 non-null object
EventDate                 2769 non-null object
WhereHeld                 2769 non-null object
Wilks                     2769 non-null float64
BodyweightKg              2769 non-null float64
Equipment                 2769 non-null int64
Sex                       2769 non-null object
Age                       2769 non-null float64
year                      2769 non-null int64
Competition_experience    2769 non-null int64
birth_year                2769 non-null float64
dtypes: float64(4), int64(4), object(4)
memory usage: 281.2+ KB


In [32]:
# third_meet_df = twenty18_kept.query('Competition_experience >= 3')
# second_meet_df = twenty18_kept.query('Competition_experience >= 2')
recent_meet_df =twenty18_kept.query('Competition_experience >= 1')

In [33]:
print(recent_meet_df.shape, model_data.shape)#, third_meet_df.shape)

(2769, 12) (12254, 22)


In [34]:
model_data = model_data[['First_Sex', 'First_Equipment', 'First_BodyweightKg', 'First_Wilks',
       'First_WhereHeld', 'First_EventDate', 'First_Competition_experience',
       'Second_Sex', 'Second_Equipment', 'Second_BodyweightKg', 'Second_Wilks',
       'Second_WhereHeld', 'Second_EventDate', 'Second_Competition_experience']]
#model_data.columns

In [35]:
column_mapping = {'Second_Equipment': 'Third_Equipment', 'Second_BodyweightKg': 'Third_BodyweightKg', 'Second_Wilks': 'Third_Wilks',
       'Second_WhereHeld': 'Third_WhereHeld', 'Second_EventDate': 'Third_EventDate', 'Second_Competition_experience': 'Third_Competition_experience',
                  'First_Equipment':'Second_Equipment', 'First_BodyweightKg': 'Second_BodyweightKg', 'First_Wilks': 'Second_Wilks', 'First_WhereHeld': 'Second_WhereHeld', 'First_EventDate': 'Second_EventDate'}


model_data.rename(columns=column_mapping, inplace=True)

In [36]:
model_data.head()

,First_Sex,Second_Equipment,Second_BodyweightKg,Second_Wilks,Second_WhereHeld,Second_EventDate,First_Competition_experience,Second_Sex,Third_Equipment,Third_BodyweightKg,Third_Wilks,Third_WhereHeld,Third_EventDate,Third_Competition_experience
AthID,,,,,,,,,,,,,,
0,1,3,100.0,418.40,SoPac,2018-01-27,1,1,1,87.20,355.37,Europe,2017-09-30,2
1,1,3,99.8,493.35,SoPac,2018-01-27,1,1,3,102.60,485.08,SoPac,2017-09-09,2
2,1,3,98.8,334.84,SoPac,2018-01-27,1,1,1,99.30,329.57,SoPac,2017-10-07,2
3,1,3,98.0,395.80,SoPac,2018-01-27,1,1,3,89.80,410.63,SoPac,2016-07-08,2
4,1,3,89.7,380.49,SoPac,2018-01-27,1,1,2,74.39,331.56,NAmerica,2017-02-23,2


In [40]:
recent_meet_df.set_index('AthID')

,Name,EventDate,WhereHeld,Wilks,BodyweightKg,Equipment,Sex,Age,year,Competition_experience,birth_year
AthID,,,,,,,,,,,
45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,0,35.0,2018,1,1983.0
42429,James Corey McManus,3/10/2018,NAmerica,206.72,77.84,1,1,37.0,2018,1,1981.0
67074,Matt Borquez,3/10/2018,NAmerica,280.78,80.38,1,1,29.0,2018,1,1989.0
9600,Avi Bagley,3/10/2018,NAmerica,363.88,79.29,1,1,28.0,2018,1,1990.0
45082,Jeff Brown,3/10/2018,NAmerica,387.28,98.52,3,1,33.0,2018,1,1985.0
87182,Sean Parisi,3/10/2018,NAmerica,327.95,93.17,3,1,20.0,2018,1,1998.0
75032,Nick Murphy,3/10/2018,NAmerica,384.66,92.26,1,1,25.0,2018,1,1993.0
82260,Robert Heaton,3/10/2018,NAmerica,356.92,95.44,1,1,41.0,2018,1,1977.0
24170,Daniel Bowers,3/10/2018,NAmerica,359.81,102.78,3,1,39.0,2018,1,1979.0


In [43]:
recent_meet_df = recent_meet_df.add_prefix('First_')
recent_meet_df.rename(columns={'First_AthID': 'AthID', 'First_Name': 'Name'}, inplace=True)
recent_meet_df.head(1)

,AthID,Name,First_EventDate,First_WhereHeld,First_Wilks,First_BodyweightKg,First_Equipment,First_Sex,First_Age,First_year,First_Competition_experience,First_birth_year
0,45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,0,35.0,2018,1,1983.0


In [42]:
recent_meet_df.head(1)

,First_AthID,First_Name,First_EventDate,First_WhereHeld,First_Wilks,First_BodyweightKg,First_Equipment,First_Sex,First_Age,First_year,First_Competition_experience,First_birth_year
0,45743,Jennifer Broome,3/10/2018,NAmerica,312.03,73.84,3,0,35.0,2018,1,1983.0


In [45]:
#recent_meet_df['AthID'].min()
df = pd.merge(model_data, recent_meet_df, on='AthID', left_index=True, right_index=True)

KeyError: 'AthID'

In [ ]:
print(df.shape, recent_meet_df.shape)

In [ ]:
df.columns

In [ ]:
data['First_Rest'] = data['First_EventDate'] - data['Second_EventDate']
data['First_Rest']=data['First_Rest'].dt.days
data['Second_Rest'] = data['Second_Rest'].dt.days